In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama3-70b-8192"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

That's an easy one!

The first person to set foot on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to walk on the moon.


In [50]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://career.tech-stack.com/front-end-engineer-v202")
page_data = loader.load().pop().page_content
print(page_data)

Front-End Engineer for TechstackJobsAbout usReviewsContact usEnUaEnUaJobsAbout usReviewsContact usWorldwide websiteJobs / Front-End EngineerApply nowProductTeamCultureResponsibilitiesRequirementsBenefitsCandidate PathCompanyRemote allowedFront-End EngineerShare:About the productIntroThe mission of our client is to accelerate the adoption of renewable energy across Europe by combining technology, innovation, and deep energy market expertise. The goal is to enable a more efficient, transparent, and scalable energy transition.Achieving this requires building intelligent platforms that connect key stakeholders across the energy ecosystem, fostering collaboration and delivering strong technical, commercial, and financial synergies to support the shift toward sustainability.Technology stackNext.js, React.jsTypeScriptVercelShadcn UI/Tailwind CSSGitHub (CI/CD)AWS LambdaAWS S3Your teamYou will become a valued member of our development team, joining a group of innovative and passionate professio

In [51]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [74]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)


if isinstance(json_res, list):
   
    result_dict = json_res[0] if json_res else {}  
elif isinstance(json_res, dict):
    result_dict = json_res  
else:
    result_dict = {}  

print(result_dict)


{'role': 'Front-End Engineer', 'experience': '4+ years', 'skills': ['React.js', 'Next.js', 'TypeScript', 'Shadcn UI', 'Tailwind CSS', 'Git', 'CI/CD pipelines', 'Agile methodologies', 'Vercel', 'AWS S3/Lambda'], 'description': 'Design, develop, and maintain web user interfaces, ensuring high performance, responsiveness, and accessibility across modern browsers and devices.'}


In [75]:
type(json_res)

list

In [76]:
jobs = json_res

In [77]:
jobs

[{'role': 'Front-End Engineer',
  'experience': '4+ years',
  'skills': ['React.js',
   'Next.js',
   'TypeScript',
   'Shadcn UI',
   'Tailwind CSS',
   'Git',
   'CI/CD pipelines',
   'Agile methodologies',
   'Vercel',
   'AWS S3/Lambda'],
  'description': 'Design, develop, and maintain web user interfaces, ensuring high performance, responsiveness, and accessibility across modern browsers and devices.'}]

In [94]:
import csv
import uuid
import chromadb

# Read CSV without pandas
data = []
with open("my_portfolio.csv", "r") as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        data.append(row)

print(f"Loaded {len(data)} rows from CSV")

# Set up ChromaDB
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

# Add documents to collection
if not collection.count():
    for i, row in enumerate(data):
        try:
            tech_stack = str(row.get("Techstack", ""))
            links = str(row.get("Links", ""))
            doc_id = str(uuid.uuid4())
            
            collection.add(
                documents=[tech_stack],
                metadatas=[{"links": links}],
                ids=[doc_id]
            )
            
            if (i+1) % 10 == 0:
                print(f"Added {i+1} documents")
                
        except Exception as e:
            print(f"Error on row {i}: {e}")
            print(f"Row data: {row}")
            break

    print("Import complete")

Loaded 20 rows from CSV


In [95]:
# Get all documents from the collection
documents = collection.get()
print(documents)

{'ids': ['b5636768-b5cb-45d6-aec4-196ad0dd0699', 'ad63f740-04c9-4a6a-a288-66673f8fb970', '604856bc-052f-4222-b654-1d1825e725d1', '2aa94857-999f-4efe-b6e7-90b93e205b7a', 'bdd2afb9-2da6-42db-9eb8-c678888fdc15', '13bdb3fb-044b-46bf-b905-8ee738e38be5', 'b0b29f15-dc25-423f-ad1d-de09ac8c7ada', '94e250d6-af3e-4f7e-860c-78a68b292800', 'c84d6732-c3aa-498f-a920-ca9867922568', '071de4fb-de2f-4b5d-8490-2157a9f5780b', '395e96b9-74c6-4792-b414-2b12ebed28ab', '16bf6a60-8a62-40b6-9fcb-6adb00907a76', '603f609f-1184-4a5e-9aaf-f32de5c8500a', 'aea2d6db-ffd4-4faa-8b9a-c01e2dd76e55', '23e66c4f-b3fe-4081-84ea-54d99ed7413d', 'a170cf16-5e6a-4ef1-8610-fe55ad1c51c0', '522ca296-df65-47e2-88c2-96676b233814', '545fce20-9191-4e27-bd36-c07c61c83ab2', '63e77536-6ee2-4e8a-af17-f282b121fb98', '6fc0c069-fd26-4a2f-a9a4-ce77cf0214d5'], 'embeddings': None, 'documents': ['React, Node.js, MongoDB', 'Angular,.NET, SQL Server', 'Vue.js, Ruby on Rails, PostgreSQL', 'Python, Django, MySQL', 'Java, Spring Boot, Oracle', 'Flutter, 

In [96]:
skills = job[0].get('skills', [])

In [97]:
links = query_results.get('metadatas', [])

In [98]:
query_results = collection.query(query_texts=skills, n_results=2)
query_results

{'ids': [['b5636768-b5cb-45d6-aec4-196ad0dd0699',
   'c84d6732-c3aa-498f-a920-ca9867922568'],
  ['545fce20-9191-4e27-bd36-c07c61c83ab2',
   '604856bc-052f-4222-b654-1d1825e725d1'],
  ['522ca296-df65-47e2-88c2-96676b233814',
   '545fce20-9191-4e27-bd36-c07c61c83ab2'],
  ['16bf6a60-8a62-40b6-9fcb-6adb00907a76',
   '13bdb3fb-044b-46bf-b905-8ee738e38be5'],
  ['a170cf16-5e6a-4ef1-8610-fe55ad1c51c0',
   '522ca296-df65-47e2-88c2-96676b233814'],
  ['6fc0c069-fd26-4a2f-a9a4-ce77cf0214d5',
   '16bf6a60-8a62-40b6-9fcb-6adb00907a76'],
  ['6fc0c069-fd26-4a2f-a9a4-ce77cf0214d5',
   '94e250d6-af3e-4f7e-860c-78a68b292800'],
  ['a170cf16-5e6a-4ef1-8610-fe55ad1c51c0',
   '6fc0c069-fd26-4a2f-a9a4-ce77cf0214d5'],
  ['a170cf16-5e6a-4ef1-8610-fe55ad1c51c0',
   '604856bc-052f-4222-b654-1d1825e725d1'],
  ['a170cf16-5e6a-4ef1-8610-fe55ad1c51c0',
   '23e66c4f-b3fe-4081-84ea-54d99ed7413d']],
 'embeddings': None,
 'documents': [['React, Node.js, MongoDB', 'React Native, Node.js, MongoDB'],
  ['Full-stack, JavaScr

In [86]:
print("Query result links:", links)

Query result links: [[{'links': 'https://example.com/react-portfolio'}, {'links': 'https://example.com/react-native-portfolio'}], [{'links': 'https://example.com/full-stack-js-portfolio'}, {'links': 'https://example.com/vue-portfolio'}], [{'links': 'https://example.com/typescript-frontend-portfolio'}, {'links': 'https://example.com/full-stack-js-portfolio'}], [{'links': 'https://example.com/kotlin-android-portfolio'}, {'links': 'https://example.com/flutter-portfolio'}], [{'links': 'https://example.com/kotlin-backend-portfolio'}, {'links': 'https://example.com/typescript-frontend-portfolio'}], [{'links': 'https://example.com/devops-portfolio'}, {'links': 'https://example.com/kotlin-android-portfolio'}], [{'links': 'https://example.com/devops-portfolio'}, {'links': 'https://example.com/magento-portfolio'}], [{'links': 'https://example.com/kotlin-backend-portfolio'}, {'links': 'https://example.com/devops-portfolio'}], [{'links': 'https://example.com/kotlin-backend-portfolio'}, {'links': '

In [87]:
flattened_links = set()  # Using a set to eliminate duplicates
for link_group in links:
    for link_item in link_group:
        flattened_links.add(link_item.get('links'))

# Convert set to list and print
flattened_links_list = list(flattened_links)
print("Unique Links:", flattened_links_list)

Unique Links: ['https://example.com/magento-portfolio', 'https://example.com/typescript-frontend-portfolio', 'https://example.com/flutter-portfolio', 'https://example.com/vue-portfolio', 'https://example.com/xamarin-portfolio', 'https://example.com/react-native-portfolio', 'https://example.com/kotlin-android-portfolio', 'https://example.com/full-stack-js-portfolio', 'https://example.com/kotlin-backend-portfolio', 'https://example.com/devops-portfolio', 'https://example.com/react-portfolio']


In [90]:
job

[{'role': 'Front-End Engineer',
  'experience': '4+ years',
  'skills': ['React.js',
   'Next.js',
   'TypeScript',
   'Shadcn UI',
   'Tailwind CSS',
   'Git',
   'CI/CD pipelines',
   'Agile methodologies',
   'Vercel',
   'AWS S3/Lambda'],
  'description': 'Design, develop, and maintain web user interfaces, ensuring high performance, responsiveness, and accessibility across modern browsers and devices.'}]

In [91]:
jobs[0].get('skills', [])

['React.js',
 'Next.js',
 'TypeScript',
 'Shadcn UI',
 'Tailwind CSS',
 'Git',
 'CI/CD pipelines',
 'Agile methodologies',
 'Vercel',
 'AWS S3/Lambda']

In [99]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Here is the cold email:

Subject: Expert Front-End Engineer for Your Web Development Needs

Dear [Client],

I came across your job posting for a Front-End Engineer and was impressed by the requirements. At AtliQ, we specialize in providing top-notch AI and software consulting services that cater to the unique needs of our clients. Our team of experts has a proven track record of delivering scalable, efficient, and cost-effective solutions.

Our Front-End Engineers possess the exact skills you're looking for, including React.js, Next.js, TypeScript, Shadcn UI, Tailwind CSS, Git, CI/CD pipelines, Agile methodologies, Vercel, and AWS S3/Lambda. We understand the importance of designing, developing, and maintaining web user interfaces that ensure high performance, responsiveness, and accessibility across modern browsers and devices.

Our portfolio showcases our expertise in Front-End development, including:

* https://example.com/react-portfolio
* https://example.com/typescript-frontend-po